In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pyarrow
from dataclasses import dataclass
import yaml
from typing import Union

In [2]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\notebooks'

In [3]:
os.chdir("..")
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01'

## Загрузка конфигураций

In [4]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

## Класс `DropBatchProcessor`

In [5]:
# ----------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС СЮДА -------------------------

**Тест `DropBatchProcessor`**

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [2]:
from data_generator.fraud.drops.build.config import DropConfigBuilder
from data_generator.fraud.drops.build.builder import DropBaseClasses
from data_generator.fraud.drops.txns import CreateDropTxn
from data_generator.fraud.drops.processor import DropBatchProcessor

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
configs = drop_cfg_build.build_dist_cfg()
# configs = drop_cfg_build.build_purch_cfg()

base_agg1 = DropBaseClasses(configs=configs)
# base_agg1.build_all(drop_type="distributor")
base_agg1.build_all(drop_type="distributor")
acc_hand1 = base_agg1.acc_hand
amt_hand1 = base_agg1.amt_hand
part_data1 = base_agg1.part_data
time_hand1 = base_agg1.time_hand
behav_hand1 = base_agg1.behav_hand

cr_drop_txn1 = CreateDropTxn(configs=configs, base=base_agg1)
drop_clients = configs.clients

batch_processor = DropBatchProcessor(base=base_agg1, create_txn=cr_drop_txn1)

for client in drop_clients.iloc[[8]].itertuples():
    part_data1.client_info = client
    acc_hand1.client_id = client.client_id
    acc_hand1.get_account(own=True)
part_data1.client_info

Pandas(Index=8, client_id=12890, district_id=70, birth_date='1958-01-20', sex='female', region='Новосибирская', area='Новосибирск', timezone='UTC+7', lat=55.0281016, lon=82.9210575, population=1498921, home_ip='2.60.20.172', geometry=<MULTIPOLYGON (((82.751 54.991, 82.751 54.991, 82.751 54.991, 82.752 54.991,...>)

In [4]:
def reset_caches(cr_drop_txn, behav_hand, amt_hand, time_hand, part_data, processor):
    cr_drop_txn.reset_cache()
    behav_hand.reset_cache(all=True)
    amt_hand.reset_cache(all=True) # batch_txns здесь
    time_hand.reset_cache()
    part_data.reset_cache()
    processor.reset_cache(all=True)

**`is_declined`**  
- cr_drop_txn1.out_txns = cr_drop_txn1.out_lim
- cr_drop_txn1.in_txns = cr_drop_txn1.in_lim
- cr_drop_txn1.out_txns = cr_drop_txn1.out_lim - 1
- cr_drop_txn1.in_txns = cr_drop_txn1.in_lim - 1

In [9]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_processor)
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim - 1
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim - 1
batch_processor.is_declined()

False

### **Тест метода `distributor`**

In [ ]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_processor)
all_txns1 = []

while True:
    declined = batch_processor.declined
    receive_txn1 = cr_drop_txn1.trf_or_atm(declined=declined, receive=receive)
    if declined:
        break
        
    batch_processor.distributor()